In this notebook, we show a full example of how the `fairret` library might be used to train a PyTorch model with a fairness cost.

# Loading some data
To start, let's load some data where fair binary classification is desirable. We'll use the `folktables` [library](https://github.com/socialfoundations/folktables) and their example data of the 2018 [American Community Survey](https://www.census.gov/programs-surveys/acs) (ACS).

In [1]:
from folktables import ACSDataSource

data_source = ACSDataSource(survey_year='2018', horizon='1-Year', survey='person')
ca_data = data_source.get_data(states=["AL"], download=True)

In [2]:
from folktables import ACSIncome, generate_categories

definition_df = data_source.get_definitions(download=True)
categories = generate_categories(features=ACSIncome.features, definition_df=definition_df)

ca_features, ca_labels, _ = ACSIncome.df_to_pandas(ca_data, categories=categories, dummies=True)
ca_features.head()

,AGEP,WKHP,"COW_Employee of a private for-profit company or business, or of an individual, for wages, salary, or commissions","COW_Employee of a private not-for-profit, tax-exempt, or charitable organization",COW_Federal government employee,"COW_Local government employee (city, county, etc.)","COW_Self-employed in own incorporated business, professional practice or farm","COW_Self-employed in own not incorporated business, professional practice, or farm",COW_State government employee,COW_Working without pay in family business or farm,...,SEX_Male,RAC1P_Alaska Native alone,RAC1P_American Indian alone,"RAC1P_American Indian and Alaska Native tribes specified; or American Indian or Alaska Native, not specified and no other races",RAC1P_Asian alone,RAC1P_Black or African American alone,RAC1P_Native Hawaiian and Other Pacific Islander alone,RAC1P_Some Other Race alone,RAC1P_Two or More Races,RAC1P_White alone
0,18,21.0,True,False,False,False,False,False,False,False,...,False,False,False,False,False,True,False,False,False,False
1,53,40.0,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
2,41,40.0,True,False,False,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True
3,18,2.0,False,False,False,False,False,True,False,False,...,False,False,False,False,False,False,False,False,False,True
4,21,50.0,False,False,True,False,False,False,False,False,...,True,False,False,False,False,False,False,False,False,True


To keep things simple for now, let's only consider two sensitive groups: *male* and *female*.

In [3]:
sens_cols = ['SEX_Female', 'SEX_Male']
feat = ca_features.drop(columns=sens_cols).to_numpy(dtype="float")
sens = ca_features[sens_cols].to_numpy(dtype="float")
label = ca_labels.to_numpy(dtype="float")

print(sens.mean(axis=0))

[0.47808514 0.52191486]


# A naive PyTorch pipeline

The `fairret` library treats sensitive features in the same way 'normal' features are treated in PyTorch: as (N x D) tensors, where N is the number of samples and D the dimensionality. In contrast to other fairness libraries you may have used, we can therefore just leave categorical sensitive features as one-hot encoded!

In [4]:
import torch
torch.manual_seed(0)
feat, sens, label = torch.tensor(feat).float(), torch.tensor(sens).float(), torch.tensor(label).float()
print(f"Shape of the 'normal' features tensor: {feat.shape}")
print(f"Shape of the sensitive features tensor: {sens.shape}")
print(f"Shape of the labels tensor: {label.shape}")

Shape of the 'normal' features tensor: torch.Size([22268, 727])
Shape of the sensitive features tensor: torch.Size([22268, 2])
Shape of the labels tensor: torch.Size([22268, 1])


In typical PyTorch fashion, let's now define a simple neural net with 1 hidden layer, an optimizer, and a DataLoader.

In [5]:
h_layer_dim = 64
lr = 1e-3
batch_size = 1024

model = torch.nn.Sequential(
    torch.nn.Linear(feat.shape[1], h_layer_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_layer_dim, 1)
)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

from torch.utils.data import TensorDataset, DataLoader
dataset = TensorDataset(feat, sens, label)
dataloader = DataLoader(dataset, batch_size=batch_size)

Now, let's train it without doing any fairness adjustment...

In [6]:
import numpy as np

nb_epochs = 10

for epoch in range(nb_epochs):
    losses = []
    for batch_feat, batch_sens, batch_target in dataloader:
        optimizer.zero_grad()
                
        logit = model(batch_feat)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(logit, batch_target)
        loss.backward()
                
        optimizer.step()
        losses.append(loss.item())
    print(f"Epoch: {epoch}, loss: {np.mean(losses)}")

Epoch: 0, loss: 0.6458532769571651
Epoch: 1, loss: 0.6078289693052118
Epoch: 2, loss: 0.573669055646116
Epoch: 3, loss: 0.5372711365873163
Epoch: 4, loss: 0.5028285180980508
Epoch: 5, loss: 0.4751153832132166
Epoch: 6, loss: 0.4543681049888784
Epoch: 7, loss: 0.43935308808630164
Epoch: 8, loss: 0.4283923831853
Epoch: 9, loss: 0.42021786895665253


# Bias analysis in fairret

Can we detect any statistical disparities (biases) in the naive model?

The `fairret` library assesses these biases by comparing a (linear-fractional) Statistic computed for each sensitive features. In our example, this is for the 'SEX_Female' and 'SEX_Male' features. For example, let's look at the positive rate (= the rate at which positives are predicted for people in each group).

In [7]:
from fairret.statistic import PositiveRate

statistic = PositiveRate()

pred = torch.sigmoid(model(feat))
stat_per_group = statistic(pred, feat, sens, label)
absolute_diff = torch.abs(stat_per_group[0] - stat_per_group[1])

print(f"The {statistic.__class__.__name__} for group {sens_cols[0]} is {stat_per_group[0]}")
print(f"The {statistic.__class__.__name__} for group {sens_cols[1]} is {stat_per_group[1]}")
print(f"The absolute difference is {torch.abs(stat_per_group[0] - stat_per_group[1])}")

The PositiveRate for group SEX_Female is 0.30833321809768677
The PositiveRate for group SEX_Male is 0.3995041251182556
The absolute difference is 0.09117090702056885


# Bias mitigation in fairret

To reduce the statistical disparity we found, we can use one of the fairrets implemented in the library. To quantify bias according to the correct statistic, we need to pass the statistic object to the fairret loss.

In [8]:
from fairret.loss import NormLoss

norm_loss = NormLoss(statistic)

Let's train another model where we now add this loss term to the objective. 

**We only need to add one line of code to the standard PyTorch training loop!**

In [9]:
nb_epochs = 10
model = torch.nn.Sequential(
    torch.nn.Linear(feat.shape[1], h_layer_dim),
    torch.nn.ReLU(),
    torch.nn.Linear(h_layer_dim, 1)
)
optimizer = torch.optim.Adam(model.parameters(), lr=lr)

for epoch in range(nb_epochs):
    losses = []
    for batch_feat, batch_sens, batch_target in dataloader:
        optimizer.zero_grad()
                
        logit = model(batch_feat)
        loss = torch.nn.functional.binary_cross_entropy_with_logits(logit, batch_target)
        loss += norm_loss(logit, batch_feat, batch_sens, batch_target)
        loss.backward()
                
        optimizer.step()
        losses.append(loss.item())
    print(f"Epoch: {epoch}, loss: {np.mean(losses)}")

Epoch: 0, loss: 0.6641122807155956
Epoch: 1, loss: 0.6309726319529794
Epoch: 2, loss: 0.6099603568965738
Epoch: 3, loss: 0.5888651555234735
Epoch: 4, loss: 0.5677005011926998
Epoch: 5, loss: 0.5519148328087546
Epoch: 6, loss: 0.5354842218485746
Epoch: 7, loss: 0.5232194336977872
Epoch: 8, loss: 0.5121059580282732
Epoch: 9, loss: 0.5073323046619241


Let's check the positive rate per group again...

In [10]:
pred = torch.sigmoid(model(feat))
stat_per_group = statistic(pred, feat, sens, label)

print(f"The {statistic.__class__.__name__} for group {sens_cols[0]} is {stat_per_group[0]}")
print(f"The {statistic.__class__.__name__} for group {sens_cols[1]} is {stat_per_group[1]}")
print(f"The absolute difference is {torch.abs(stat_per_group[0] - stat_per_group[1])}")

The PositiveRate for group SEX_Female is 0.30945485830307007
The PositiveRate for group SEX_Male is 0.31116288900375366
The absolute difference is 0.0017080307006835938


With a small change, the absolute difference between the statistics was reduced from 9.1% to 0.17% !

Though this was a simple example, it illustrates how powerful the `fairret` paradigm can be. 

Feel free to go back and try out some other statistics to compare or fairret losses to minimize. Both are designed to be easily interchangeable and extensible.

# Further examples

We plan to add more examples, including with multiple sensitive attributes. Check back later!